In [58]:
!pip install bs4

You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [59]:
import requests
import urllib.request
import datetime
from bs4 import BeautifulSoup
import pandas as pd


In [60]:
country = 'Canada'
src_cat = 'Government Website'
columns = ['start_date', 'country', 'region', 'subregion', 'source_url', 'source_category', 'source_title', 'source_full_text']

# Ontario

Since Ontario shows the most recent news on the first page, the range
will need to continue to be expanded to capture all posts. Generally ~4 pages capture ~2 weeks.

In [63]:
targets = ['https://news.ontario.ca/archive/en?page=' + str(i) for i in range(1,4)]

region = 'Ontario'
subregion = ''

# Specific structure for news.contario.ca/archive
links = []
for target in targets:
    if target.startswith('https://news.ontario.ca/archive/'):
        response = requests.get(target)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.findAll('article')
        
        for article in articles:
            smallersoup = BeautifulSoup(str(article), "html.parser")
            link = smallersoup.findAll('a')[0]['href']
            title = smallersoup.findAll('a')[0].string
            pub_date = smallersoup.time['datetime']
            
            response = requests.get(link)
            linksoup = BeautifulSoup(response.text, "html.parser")
            full_text = linksoup.article.text
            
            row = [pub_date, country, region, subregion, link, src_cat, title, full_text]
            links.append(row)
            
            # Get this link and copy full text


In [64]:
df = pd.DataFrame(links, columns=columns)

In [65]:
df.shape

(45, 8)

In [66]:
df.to_csv('sources/ontario.csv')

# Manitoba

Retrieve all news releases in 2020 for the Province of Manitoba.

In [67]:
url_base = 'https://news.gov.mb.ca'
targets = [url_base + '/news/index.html?month=' + str(i) + '&year=2020&day=01&bgnG=GO&d=' for i in range(1,12)]


In [70]:
%%time

region = 'Manitoba'
subregion = ''

links = []
for target in targets:
    print(target)
    if target.startswith(url_base): #manitoba
        response = requests.get(target)
        soup = BeautifulSoup(response.text, "html.parser")
        items = soup.findAll("div", {"class": "maincontent"})
        smallersoup = BeautifulSoup(str(items), "html.parser")
        for article in smallersoup.findAll('h2'):
            a = article.a
            relative_link = a['href']
            link = url_base + relative_link.split('..')[-1]
            title = a.string
        
            response = requests.get(link)
            linksoup = BeautifulSoup(response.text, "html.parser")
            
            date_text = linksoup.findAll("span", {"class": "article_date"})[0].string
            date = pd.to_datetime(date_text, format='%B %d, %Y')
            pub_date = date.strftime('%m/%d/%Y')
            
            full_text = linksoup.findAll("div", {"class": ""})[0].text

            
            row = [pub_date, country, region, link, src_cat, title, full_text]
            links.append(row)
            
            # Get this link and copy full text

https://news.gov.mb.ca/news/index.html?month=1&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=2&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=3&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=4&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=5&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=6&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=7&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=8&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=9&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=10&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=11&year=2020&day=01&bgnG=GO&d=
CPU times: user 13.3 s, sys: 374 ms, total: 13.7 s
Wall time: 3min 44s


In [47]:
df = pd.DataFrame(links, columns=columns)
df.to_csv('sources/manitoba.csv')
df.shape

(260, 4)